In [1]:
import math, re, os, warnings, random
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

E0000 00:00:1718690933.228622      12 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="localhost"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
D0618 06:08:53.236913327      12 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D0618 06:08:53.236928817      12 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D0618 06:08:53.236932064      12 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D0618 06:08:53.236934469      12 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D0618 06:08:53.236936872      12 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (

Tensorflow version 2.16.1
Device: 
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1718690946.914168      12 service.cc:145] XLA service 0x5aec67498320 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1718690946.914229      12 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1718690946.914234      12 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1718690946.914237      12 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1718690946.914240      12 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1718690946.914243      12 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1718690946.914245      12 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1718690946.914248      12 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1718690946.914250      12 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [2]:
!pip install plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 24.8 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow" # you can also use tensorflow or torch
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.9" # avoid memory fragmentation on JAX backend.
import plotly.graph_objs as go
import plotly.express as px
from IPython.display import display, Markdown
import tqdm as notebook_tqdm

import keras
import keras_nlp

import numpy as np
import pandas as pd


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df1 = pd.read_csv("/kaggle/input/math-qsa-dataset/train.csv")
df2 = pd.read_csv("/kaggle/input/math-qsa-dataset/test.csv")
df = pd.concat([df1, df2], axis=0)

In [5]:
def is_integer(text):
    try:
        if int(text) >= 0:
            return True
        else:
            return False
    except ValueError:
        return False
    
df["is_integer"] = df.answer.map(is_integer)
df = df[df.is_integer].reset_index(drop=True)

# Preprocessing Pipeline

In [6]:
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.6 MB/s eta 0:00:0000:010:01

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [7]:
import re
import nltk
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

# Ensure the necessary NLTK data files are downloaded
nltk.download('punkt')
nltk.download('stopwords')

class Preprocessing:
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))

    def convert_draw_command(self, draw_command):
        pattern_pentagon = re.compile(r'draw\(\((.*?)\)--\((.*?)\)--\((.*?)\)--\((.*?)\)--\((.*?)\)--\((.*?)\)--cycle.*?\);')
        match_pentagon = pattern_pentagon.match(draw_command)
        if match_pentagon:
            coords = match_pentagon.groups()
            return f"A regular pentagon with vertices at {coords[0]}, {coords[1]}, {coords[2]}, {coords[3]}, and {coords[4]}."
        
        pattern_hexagon = re.compile(r'draw\(\((.*?)\)--\((.*?)\)--\((.*?)\)--\((.*?)\)--\((.*?)\),.*?\);')
        match_hexagon = pattern_hexagon.match(draw_command)
        if match_hexagon:
            coords = match_hexagon.groups()
            return f"A regular hexagon with vertices at {coords[0]}, {coords[1]}, {coords[2]}, {coords[3]}, and {coords[4]}."
        
        return ""

    def convert_dot_label_commands(self, text):
        pattern_dot = re.compile(r'dot\(\((.*?)\)\);')
        text = pattern_dot.sub(r'A point at \1.', text)
        
        pattern_label = re.compile(r'label\("(.*?)",\((.*?)\),.*?\);')
        text = pattern_label.sub(r'The point \1 is at coordinates \2.', text)
        
        return text

    def preprocess_text(self, text):
        # Remove the [asy] tags
        text = re.sub(r'\[asy\]', '', text)
        text = re.sub(r'\[\/asy\]', '', text)

        # Split the text into commands
        commands = text.split('\n')

        readable_text = []
        for command in commands:
            if 'draw' in command:
                readable_text.append(self.convert_draw_command(command))
            else:
                readable_text.append(self.convert_dot_label_commands(command))

        readable_text = ' '.join(readable_text)

        # Tokenize into sentences
        sentences = sent_tokenize(readable_text)

        # Remove stop words and tokenize the remaining words
        filtered_sentences = []
        for sentence in sentences:
            word_tokens = word_tokenize(sentence)
            filtered_sentence = [word for word in word_tokens if word.lower() not in self.stop_words]
            filtered_sentences.append(' '.join(filtered_sentence))

        filtered_text = ' '.join(filtered_sentences)
        return filtered_text

    def process_dataframe(self, df, text_column):
        df[f'{text_column}'] = df[text_column].apply(self.preprocess_text)
        return df



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Prompt Engineering

In [8]:
template = """Role:\nYou are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.\n\nInstruction:
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process. Keep in mind that answer must be a non-negative integer number.
3. At the end, create a "Answer" section where you will state only the final numerical or algebraic answer, without any additional text or narrative.\n\nProblem:\n{problem}\n\nSolution:\n{solution}"""

In [9]:
from tqdm.notebook import tqdm
tqdm.pandas() # progress bar for pandas
df["prompt"] = df.apply(lambda row: template.format(problem=row.problem,
                                                             solution=f"{row.solution}\n\nAnswer:\n{row.answer}"),
                                                             axis=1)
data = df.prompt.tolist()

In [10]:
def colorize_text(text):
    for word, color in zip(["Role", "Instruction", "Problem", "Solution", "Answer"],
                           ["blue", "yellow", "red", "cyan", "green"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

## Model Training

In [11]:
def split_text_into_prompt_completion(df, text_column):
    prompts = []
    completions = []

    for index, row in df.iterrows():
        text = row[text_column]
        
        # Split based on "Solution:"
        problem_part, solution_part = text.split("Solution:", 1)
        
        # Ensure to keep the "Solution:" keyword in the completion
        solution_part = "Solution:" + solution_part
        
        # Append to lists
        prompts.append(problem_part.strip())
        completions.append(solution_part.strip())
    
    # Create new DataFrame
    split_df = pd.DataFrame({
        "prompt": prompts,
        "completion": completions
    })
    
    return split_df

# Example usage
data = split_text_into_prompt_completion(df, 'prompt')

In [12]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras_nlp.models import Llama3CausalLM, Llama3Tokenizer

prompts = data['prompt'].tolist()
results = data['completion'].tolist()

# Load the model and tokenizer
llama_lm = Llama3CausalLM.from_preset("llama3_8b_en", dtype="bfloat16")
tokenizer = Llama3Tokenizer.from_preset("llama3_8b_en")

Attaching 'metadata.json' from model 'keras/llama3/keras/llama3_8b_en/3' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/llama3/keras/llama3_8b_en/3' to your Kaggle notebook...
Attaching 'task.json' from model 'keras/llama3/keras/llama3_8b_en/3' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/llama3/keras/llama3_8b_en/3' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/llama3/keras/llama3_8b_en/3' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/llama3/keras/llama3_8b_en/3' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/llama3/keras/llama3_8b_en/3' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/llama3/keras/llama3_8b_en/3' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/llama3/keras/llama3_8b_en/3' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/llama3/keras/llama3_8b_en/3' to your Kaggle notebook...
Attachi

In [33]:
tokenized_prompts = tokenizer(prompts)
tokenized_results = tokenizer(results)


max_len = max(max(len(seq) for seq in tokenized_prompts), max(len(seq) for seq in tokenized_results))
print(max_len)
# Function to pad sequences

2412


In [40]:
def pad_sequences(sequences, max_len, pad_token_id):
    padded_sequences = np.full((sequences.shape[0], max_len), pad_token_id, dtype=np.int32)
    for i, seq in enumerate(sequences):
        padded_sequences[i, :len(seq)] = seq
    return padded_sequences

In [41]:
# Pad the tokenized sequences
pad_token_id = tokenizer.pad_token_id
print(pad_token_id)

0


In [42]:
tokenized_prompts.shape

TensorShape([7356, None])

In [43]:

input_ids = pad_sequences(tokenized_prompts, max_len, pad_token_id)
output_ids = pad_sequences(tokenized_results, max_len, pad_token_id)


# Create a TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((tokenized_prompts, tokenized_results))
batch_size = 8  # Adjust according to your TPU capacity
dataset = dataset.shuffle(len(prompts)).batch(batch_size)

# Define training configuration
learning_rate = 1e-5
optimizer = tf.keras.optimizers.Adam(learning_rate)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
llama_lm.compile(optimizer=optimizer, loss=loss)

# Train the model
epochs = 3  # Adjust according to your needs
llama_lm.fit(dataset, epochs=epochs)

ValueError: in user code:

    File "/usr/local/lib/python3.10/site-packages/keras_nlp/src/models/task.py", line 98, in preprocess_samples  *
        return self.preprocessor(x, y=y, sample_weight=sample_weight)
    File "/usr/local/lib/python3.10/site-packages/keras_nlp/src/layers/preprocessing/preprocessing_layer.py", line 49, in __call__  *
        outputs = super().__call__(*args, **kwargs)
    File "/usr/local/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 122, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/site-packages/keras_nlp/src/models/llama3/llama3_causal_lm_preprocessor.py", line 111, in call
        x = convert_inputs_to_list_of_tensor_segments(x)[0]
    File "/usr/local/lib/python3.10/site-packages/keras_nlp/src/utils/keras_utils.py", line 100, in convert_inputs_to_list_of_tensor_segments
        raise ValueError(

    ValueError: Exception encountered when calling Llama3CausalLMPrepr